In [19]:
from docx import Document
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
def read_doc(file_path, patent_start=r'^\s*US[0-9A-Z]*$'):
    doc = Document(file_path)
    labels = []
    patents = []
    current_patent_text = []
    
    for i, paragraph in enumerate(doc.paragraphs):
        if re.search(patent_start, paragraph.text):
            labels.append(paragraph.text)
            if current_patent_text:
                patents.append('\n'.join(current_patent_text))
            current_patent_text = []
        else:
            # If not a patent start, add the paragraph text to the current patent text
            current_patent_text.append(paragraph.text)
            
    # Add the last patent text
    if current_patent_text:
        patents.append('\n'.join(current_patent_text))

    return patents, labels

In [21]:
def preprocess_text(text):
    stop_words = stopwords.words('english')
    stop_words.append('may')
    stop_words = set(stop_words)
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [22]:
patents, labels = read_doc(r'C:\Users\johnt\Downloads\20240106021625210-AMD (2).docx')
for i in range(len(patents)):
    patents[i] = preprocess_text(patents[i])

In [50]:
tagged_data = [TaggedDocument(words=doc.split(), tags=[label]) for label, doc in zip(labels, patents)]

In [51]:
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [53]:
inferred_vectors = [model.infer_vector(doc.words) for doc in tagged_data]

In [60]:
similar_documents = [model.dv.most_similar([inferred_vector], topn=5) for inferred_vector in inferred_vectors]

In [80]:
threshold = 0.8
keywords_per_doc = []

In [82]:
for doc_id, inferred_vector in enumerate(inferred_vectors):
    word_weights = model.dv[doc_id]

    sorted_words = sorted(model.wv.index_to_key, key=lambda word: word_weights[model.wv.key_to_index[word]], reverse=True)


IndexError: index 100 is out of bounds for axis 0 with size 100

In [79]:
keywords_per_doc[0]

(' US8053849B2', [' US8053849B2', ' US8753943B2', ' US8445975B2'])